In [1]:
import imp
from bins import Bins
from utils import calcAllChi2, makeHists

bins = Bins.readFrom("ranges.yml")
histsMC = makeHists("medium_isotropic_eff_ag1230ag_nn_9deg.dat", "_MC", bins)
histsData = makeHists("apr12_diele_088_090_ag123ag_2500A_accepted_nn_2.dat", "_data", bins)

calcAllChi2(histsMC, histsData)

/tmp/ipykernel_30811/563685925.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/szymon/Downloads/yes/envs/pyroot/etc/cling/std.modulemap:257:29: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "memory_resource" [optional] {
                            ^
/home/szymon/Downloads/yes/envs/pyroot/etc/cling/std.modulemap:447:27: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "bits/chrono.h" [optional] {
                          ^
/home/szymon/Downloads/yes/envs/pyroot/etc/cling/std.modulemap:531:41: warning: unknown attribute 'optional' [-Wignored-attributes]
  explicit module "bits_ranges_base_h" [optional] {
                                        ^
/home/szymon/Downloads/yes/envs/pyroot/etc/cling/std.modulemap:538:32: warning: unknown attribute 'optional' [-Wignored-attributes]
  module "bits/ranges_util.h" [optional] {
                        

Welcome to JupyROOT 6.28/04
histname:  hist_0to150_m06m02_MC
histname:  hist_0to150_m02p01_MC
histname:  hist_0to150_p01p08_MC
histname:  hist_150to300_m06m02_MC
histname:  hist_150to300_m02p01_MC
histname:  hist_150to300_p01p08_MC
histname:  hist_300to450_m06m02_MC
histname:  hist_300to450_m02p01_MC
histname:  hist_300to450_p01p08_MC
histname:  hist_450to600_m06m02_MC
histname:  hist_450to600_m02p01_MC
histname:  hist_450to600_p01p08_MC
histname:  hist_0to150_m06m02_data
histname:  hist_0to150_m02p01_data
histname:  hist_0to150_p01p08_data
histname:  hist_150to300_m06m02_data
histname:  hist_150to300_m02p01_data
histname:  hist_150to300_p01p08_data
histname:  hist_300to450_m06m02_data
histname:  hist_300to450_m02p01_data
histname:  hist_300to450_p01p08_data
histname:  hist_450to600_m06m02_data
histname:  hist_450to600_m02p01_data
histname:  hist_450to600_p01p08_data
chi2, ndf, chi2/ndf 33.74570066533127 36 0.9373805740369797
chi2, ndf, chi2/ndf 78.9408645337891 123 0.6417956466161715


In [2]:
import numpy as np
import torch

torch.manual_seed(0)

import gpytorch
import botorch

import matplotlib.pyplot as plt

plt.style.use("bmh")
plt.rcParams["figure.figsize"] = (8, 6)

from tqdm.notebook import tqdm

import warnings

/home/szymon/Downloads/yes/envs/pyroot/lib/python3.8/site-packages/ROOT/_facade.py:154: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  return _orig_ihook(name, *args, **kwds)


In [ ]:
lb = -1
ub = 1

bounds = torch.tensor([[lb], [ub]], dtype=torch.float)

In [ ]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=1)
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [ ]:
num_queries = 10
num_repeats = 10

In [ ]:
strategy = "cei"  # "cei" or "ei"
strategy = strategy.upper()

# -2 is the default value when no feasible has been found
default_value = -2
feasible_incumbents = torch.ones((num_repeats, num_queries)) * default_value

for trial in range(num_repeats):
    print("trial", trial)

    torch.manual_seed(trial)
    train_x = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(1, 1)
    train_utility = objective(train_x)
    train_cost = cost(train_x)

    for i in tqdm(range(num_queries)):
        feasible_flag = (train_cost <= 0).any()

        if feasible_flag:
            feasible_incumbents[trial, i] = train_utility[train_cost <= 0].max()

        utility_model, utility_likelihood = fit_gp_model(
            train_x, train_utility.squeeze(-1)
        )
        cost_model, cost_likelihood = fit_gp_model(train_x, train_cost.squeeze(-1))

        if feasible_flag:
            best_f = train_utility[train_cost <= 0].max()
        else:
            best_f = torch.tensor(default_value)

        if strategy == "EI":
            policy = botorch.acquisition.analytic.ExpectedImprovement(
                model=utility_model,
                best_f=train_utility.max(),
            )

        if strategy == "CEI":
            policy = botorch.acquisition.analytic.ConstrainedExpectedImprovement(
                model=botorch.models.model_list_gp_regression.ModelListGP(
                    utility_model, cost_model
                ),
                best_f=best_f,
                objective_index=0,
                constraints={1: [None, 0]}
            )
        
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=RuntimeWarning)
            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

        next_utility = objective(next_x)
        next_cost = cost(next_x)

        train_x = torch.cat([train_x, next_x])
        train_utility = torch.cat([train_utility, next_utility])
        train_cost = torch.cat([train_cost, next_cost])

In [ ]:
torch.save(feasible_incumbents, f"./{strategy}.pth")